#**Install Library**

In [1]:
!pip install basemap-data-hires
!pip install seaborn
!pip install basemap numpy pandas scipy
!pip install cartopy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.1/91.1 MB 8.9 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 935.8/935.8 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 42.4 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 79.4 MB/s eta 0:00:00


#**Import Library**

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
from mpl_toolkits.basemap import Basemap
from matplotlib.colors import LogNorm, Normalize
from scipy.stats import linregress
import datetime
from datetime import datetime
import seaborn as sns


#**Read Dataset**

In [3]:
dataset = "https://raw.githubusercontent.com/arifadli/DatasetRepository/main/Disertation/Indonesia/QueryCountryIndonesia(1910-2016)USGS.csv"
df = pd.read_csv(dataset)
df.head(5)


,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2016-12-31T17:22:51.700Z,1.0439,126.5326,42.38,4.0,mb,NaN,197.0,0.876,0.98,...,2017-03-23T22:52:11.040Z,"97km WNW of Kota Ternate, Indonesia",earthquake,11.8,27.3,0.183,8.0,reviewed,us,us
1,2016-12-31T07:21:08.220Z,-6.0786,130.0318,137.85,4.6,mb,NaN,82.0,2.271,0.86,...,2017-03-23T22:52:10.040Z,"250km NW of Saumlaki, Indonesia",earthquake,8.9,8.2,0.122,20.0,reviewed,us,us
2,2016-12-30T08:24:53.620Z,-9.1767,123.5689,116.02,4.6,mb,NaN,59.0,0.897,0.89,...,2017-03-23T22:52:09.040Z,"51km NNW of Naisano Dua, Indonesia",earthquake,7.7,6.6,0.110,25.0,reviewed,us,us
3,2016-12-30T07:10:05.390Z,1.7358,126.5028,42.15,5.0,mb,NaN,92.0,1.289,0.95,...,2017-03-23T22:52:09.040Z,"142km NW of Kota Ternate, Indonesia",earthquake,7.8,7.4,0.075,57.0,reviewed,us,us
4,2016-12-30T06:41:20.480Z,-9.1597,118.7325,105.58,4.4,mb,NaN,81.0,3.502,0.64,...,2017-03-23T22:52:09.040Z,"45km SSE of Tolotangga, Indonesia",earthquake,6.7,8.6,0.134,16.0,reviewed,us,us


In [4]:
# Membaca dataset
dataset = "https://raw.githubusercontent.com/arifadli/DatasetRepository/main/Disertation/Indonesia/QueryCountryIndonesia(1910-2016)USGS.csv"
df = pd.read_csv(dataset)

# Mengonversi kolom 'time' menjadi tipe datetime
df['time'] = pd.to_datetime(df['time'])

# Menerapkan filter untuk tahun 1966 - 2016
df_filtered = df[(df['time'].dt.year >= 1966) & (df['time'].dt.year <= 2016)]

# Menampilkan DataFrame yang telah difilter
df = df_filtered
df.head(10)

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2016-12-31 17:22:51.700000+00:00,1.0439,126.5326,42.38,4.0,mb,NaN,197.0,0.876,0.98,...,2017-03-23T22:52:11.040Z,"97km WNW of Kota Ternate, Indonesia",earthquake,11.8,27.3,0.183,8.0,reviewed,us,us
1,2016-12-31 07:21:08.220000+00:00,-6.0786,130.0318,137.85,4.6,mb,NaN,82.0,2.271,0.86,...,2017-03-23T22:52:10.040Z,"250km NW of Saumlaki, Indonesia",earthquake,8.9,8.2,0.122,20.0,reviewed,us,us
2,2016-12-30 08:24:53.620000+00:00,-9.1767,123.5689,116.02,4.6,mb,NaN,59.0,0.897,0.89,...,2017-03-23T22:52:09.040Z,"51km NNW of Naisano Dua, Indonesia",earthquake,7.7,6.6,0.110,25.0,reviewed,us,us
3,2016-12-30 07:10:05.390000+00:00,1.7358,126.5028,42.15,5.0,mb,NaN,92.0,1.289,0.95,...,2017-03-23T22:52:09.040Z,"142km NW of Kota Ternate, Indonesia",earthquake,7.8,7.4,0.075,57.0,reviewed,us,us
4,2016-12-30 06:41:20.480000+00:00,-9.1597,118.7325,105.58,4.4,mb,NaN,81.0,3.502,0.64,...,2017-03-23T22:52:09.040Z,"45km SSE of Tolotangga, Indonesia",earthquake,6.7,8.6,0.134,16.0,reviewed,us,us
5,2016-12-30 06:06:34.030000+00:00,0.4049,123.7758,211.57,4.6,mb,NaN,55.0,1.753,0.77,...,2017-03-23T22:52:09.040Z,"23km W of Molibagu, Indonesia",earthquake,6.6,7.2,0.099,30.0,reviewed,us,us
6,2016-12-29 22:30:19.300000+00:00,-9.0279,118.6639,79.00,6.3,mww,NaN,26.0,3.553,1.42,...,2017-03-23T22:52:09.040Z,"29km S of Tolotangga, Indonesia",earthquake,6.2,1.8,NaN,NaN,reviewed,us,us
7,2016-12-29 14:37:33.160000+00:00,3.9723,126.7909,82.30,4.3,mb,NaN,177.0,3.230,0.83,...,2017-03-23T22:52:08.040Z,"216km SE of Sarangani, Philippines",earthquake,5.3,14.5,0.177,9.0,reviewed,us,us
8,2016-12-29 08:16:44.850000+00:00,-4.6133,101.7366,31.98,4.3,mb,NaN,112.0,1.241,0.62,...,2017-03-23T22:52:08.040Z,"107km SSW of Bengkulu, Indonesia",earthquake,5.2,6.8,0.147,13.0,reviewed,us,us
9,2016-12-29 01:22:26.280000+00:00,-4.7053,101.6970,31.82,4.3,mb,NaN,207.0,1.300,0.35,...,2017-03-23T22:52:08.040Z,"118km SSW of Bengkulu, Indonesia",earthquake,10.8,5.2,0.217,6.0,reviewed,us,us


In [5]:
# URL dataset
dataset_url = "https://raw.githubusercontent.com/arifadli/DatasetRepository/main/Disertation/Indonesia/QueryCountryIndonesia(1910-2016)USGS.csv"

# Membaca dataset CSV
df = pd.read_csv(dataset_url)

# Mengonversi kolom "time" ke format datetime
df['time'] = pd.to_datetime(df['time'])

# Mengekstrak tahun dari kolom "time" dan menambahkannya sebagai kolom baru "year"
df['year'] = df['time'].dt.year

# Menampilkan lima baris pertama dengan kolom "year"
print(df[['time', 'year']].head())

                              time  year
0 2016-12-31 17:22:51.700000+00:00  2016
1 2016-12-31 07:21:08.220000+00:00  2016
2 2016-12-30 08:24:53.620000+00:00  2016
3 2016-12-30 07:10:05.390000+00:00  2016
4 2016-12-30 06:41:20.480000+00:00  2016


In [6]:
# Mengambil hanya kolom yang diinginkan
selected_columns = ['time', 'latitude', 'longitude', 'place','depth', 'mag', 'magType', 'net', 'id', 'updated', 'type', 'status', 'locationSource', 'magSource','year']
df_selected_columns = df[selected_columns]

# Menampilkan DataFrame yang hanya berisi kolom yang dipilih
df_selected_columns.head(10)
df = df_selected_columns
df

,time,latitude,longitude,place,depth,mag,magType,net,id,updated,type,status,locationSource,magSource,year
0,2016-12-31 17:22:51.700000+00:00,1.0439,126.5326,"97km WNW of Kota Ternate, Indonesia",42.38,4.0,mb,us,us10007tyr,2017-03-23T22:52:11.040Z,earthquake,reviewed,us,us,2016
1,2016-12-31 07:21:08.220000+00:00,-6.0786,130.0318,"250km NW of Saumlaki, Indonesia",137.85,4.6,mb,us,us10007nv4,2017-03-23T22:52:10.040Z,earthquake,reviewed,us,us,2016
2,2016-12-30 08:24:53.620000+00:00,-9.1767,123.5689,"51km NNW of Naisano Dua, Indonesia",116.02,4.6,mb,us,us10007nqa,2017-03-23T22:52:09.040Z,earthquake,reviewed,us,us,2016
3,2016-12-30 07:10:05.390000+00:00,1.7358,126.5028,"142km NW of Kota Ternate, Indonesia",42.15,5.0,mb,us,us10007npz,2017-03-23T22:52:09.040Z,earthquake,reviewed,us,us,2016
4,2016-12-30 06:41:20.480000+00:00,-9.1597,118.7325,"45km SSE of Tolotangga, Indonesia",105.58,4.4,mb,us,us10007tqx,2017-03-23T22:52:09.040Z,earthquake,reviewed,us,us,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54915,1914-10-23 06:18:44+00:00,4.2390,128.1410,"north of Halmahera, Indonesia",35.00,7.4,mw,iscgem,iscgem914014,2015-05-13T18:52:41.000Z,earthquake,automatic,iscgem,iscgem,1914
54916,1914-06-25 19:07:25+00:00,-3.9240,101.8200,"southern Sumatra, Indonesia",35.00,7.6,mw,iscgem,iscgem913990,2015-05-13T18:52:41.000Z,earthquake,automatic,iscgem,iscgem,1914
54917,1914-05-26 14:22:47+00:00,-1.8290,136.9430,"Biak region, Indonesia",15.00,8.1,mw,iscgem,iscgem913984,2015-05-13T18:52:41.000Z,earthquake,automatic,iscgem,iscgem,1914
54918,1913-03-14 08:45:04+00:00,5.3540,126.1210,"Mindanao, Philippines",15.00,7.8,mw,iscgem,iscgem914043,2015-05-13T18:52:41.000Z,earthquake,automatic,iscgem,iscgem,1913


In [7]:
# Menerapkan filter untuk tahun 1966 - 2016
df_filtered = df[(df['time'].dt.year >= 1966) & (df['time'].dt.year <= 2016)]
df_filtered

,time,latitude,longitude,place,depth,mag,magType,net,id,updated,type,status,locationSource,magSource,year
0,2016-12-31 17:22:51.700000+00:00,1.0439,126.5326,"97km WNW of Kota Ternate, Indonesia",42.38,4.0,mb,us,us10007tyr,2017-03-23T22:52:11.040Z,earthquake,reviewed,us,us,2016
1,2016-12-31 07:21:08.220000+00:00,-6.0786,130.0318,"250km NW of Saumlaki, Indonesia",137.85,4.6,mb,us,us10007nv4,2017-03-23T22:52:10.040Z,earthquake,reviewed,us,us,2016
2,2016-12-30 08:24:53.620000+00:00,-9.1767,123.5689,"51km NNW of Naisano Dua, Indonesia",116.02,4.6,mb,us,us10007nqa,2017-03-23T22:52:09.040Z,earthquake,reviewed,us,us,2016
3,2016-12-30 07:10:05.390000+00:00,1.7358,126.5028,"142km NW of Kota Ternate, Indonesia",42.15,5.0,mb,us,us10007npz,2017-03-23T22:52:09.040Z,earthquake,reviewed,us,us,2016
4,2016-12-30 06:41:20.480000+00:00,-9.1597,118.7325,"45km SSE of Tolotangga, Indonesia",105.58,4.4,mb,us,us10007tqx,2017-03-23T22:52:09.040Z,earthquake,reviewed,us,us,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54462,1966-03-08 05:41:06+00:00,1.9030,126.3340,Molucca Sea,35.00,6.2,mw,iscgem,iscgem849033,2015-05-13T18:53:00.000Z,earthquake,automatic,iscgem,iscgem,1966
54463,1966-02-13 06:36:01+00:00,-6.5610,132.6420,"Kepulauan Tanimbar, Indonesia",35.00,6.0,mw,iscgem,iscgem850145,2015-05-13T18:53:00.000Z,earthquake,automatic,iscgem,iscgem,1966
54464,1966-02-09 07:18:50+00:00,-9.8510,116.3760,"Lombok region, Indonesia",45.00,5.8,mw,iscgem,iscgem850037,2015-05-13T18:53:00.000Z,earthquake,automatic,iscgem,iscgem,1966
54465,1966-02-03 05:48:11+00:00,0.2110,123.5550,"Minahasa, Sulawesi, Indonesia",158.60,5.7,mw,iscgem,iscgem849825,2015-05-13T18:53:00.000Z,earthquake,automatic,iscgem,iscgem,1966


In [8]:
df = df_filtered

In [9]:
def bvalue_OLS(magnitude_series):
    magnitudes = magnitude_series.value_counts().sort_index()

    M = magnitudes.index.tolist()
    values = magnitudes.tolist()
    N = np.cumsum(values[::-1])[::-1]
    logN = np.log10(N)
    b,a = np.polyfit(M, logN, 1)
    return b*(-1)

def avalue_OLS(magnitude_series):
    magnitudes = magnitude_series.value_counts().sort_index()

    M = magnitudes.index.tolist()
    values = magnitudes.tolist()
    N = np.cumsum(values[::-1])[::-1]
    logN = np.log10(N)
    b,a = np.polyfit(M, logN, 1)
    return a

def generate_GR_Adeli_Panakkat_features(earthquake_dataset, time_window):
    earthquake_dataset['b-value'] = earthquake_dataset['mag'].rolling(time_window).apply(func=bvalue_OLS,raw = False).shift(1)
    earthquake_dataset['a-value'] = earthquake_dataset['mag'].rolling(time_window).apply(func=avalue_OLS,raw = False).shift(1)
    earthquake_dataset['Max magnitude expected'] = earthquake_dataset['a-value']/earthquake_dataset['b-value']

    return earthquake_dataset

In [10]:
indonesia_region = df[df.mag >= 4.0].reset_index(drop = True)

In [11]:
indonesia_region = generate_GR_Adeli_Panakkat_features(indonesia_region, 50)

In [12]:
indonesia_region.tail()

,time,latitude,longitude,place,depth,mag,magType,net,id,updated,type,status,locationSource,magSource,year,b-value,a-value,Max magnitude expected
51144,1966-03-08 05:41:06+00:00,1.903,126.334,Molucca Sea,35.0,6.2,mw,iscgem,iscgem849033,2015-05-13T18:53:00.000Z,earthquake,automatic,iscgem,iscgem,1966,0.846806,6.385604,7.540809
51145,1966-02-13 06:36:01+00:00,-6.561,132.642,"Kepulauan Tanimbar, Indonesia",35.0,6.0,mw,iscgem,iscgem850145,2015-05-13T18:53:00.000Z,earthquake,automatic,iscgem,iscgem,1966,0.852336,6.427646,7.541209
51146,1966-02-09 07:18:50+00:00,-9.851,116.376,"Lombok region, Indonesia",45.0,5.8,mw,iscgem,iscgem850037,2015-05-13T18:53:00.000Z,earthquake,automatic,iscgem,iscgem,1966,0.857352,6.463244,7.538610
51147,1966-02-03 05:48:11+00:00,0.211,123.555,"Minahasa, Sulawesi, Indonesia",158.6,5.7,mw,iscgem,iscgem849825,2015-05-13T18:53:00.000Z,earthquake,automatic,iscgem,iscgem,1966,0.853431,6.429364,7.533547
51148,1966-01-11 03:10:54+00:00,0.586,120.141,"Minahasa, Sulawesi, Indonesia",35.0,5.9,mw,iscgem,iscgem850772,2015-05-13T18:53:00.000Z,earthquake,automatic,iscgem,iscgem,1966,0.846995,6.373494,7.524829


#**Variasi Spasial nilai a**

In [13]:
# Load dataset
fixkolom = indonesia_region

# Ekstrak nama kota dari kolom 'place'
fixkolom['city'] = fixkolom['place'].str.split(',').str[0].str.split('of').str[-1].str.strip()

In [14]:
#reduksi kolom
fixkolom = indonesia_region[['time','latitude', 'longitude', 'place','city','depth', 'mag','magType','year','a-value','b-value']]
fixkolom.head(10)

,time,latitude,longitude,place,city,depth,mag,magType,year,a-value,b-value
0,2016-12-31 17:22:51.700000+00:00,1.0439,126.5326,"97km WNW of Kota Ternate, Indonesia",Kota Ternate,42.38,4.0,mb,2016,NaN,NaN
1,2016-12-31 07:21:08.220000+00:00,-6.0786,130.0318,"250km NW of Saumlaki, Indonesia",Saumlaki,137.85,4.6,mb,2016,NaN,NaN
2,2016-12-30 08:24:53.620000+00:00,-9.1767,123.5689,"51km NNW of Naisano Dua, Indonesia",Naisano Dua,116.02,4.6,mb,2016,NaN,NaN
3,2016-12-30 07:10:05.390000+00:00,1.7358,126.5028,"142km NW of Kota Ternate, Indonesia",Kota Ternate,42.15,5.0,mb,2016,NaN,NaN
4,2016-12-30 06:41:20.480000+00:00,-9.1597,118.7325,"45km SSE of Tolotangga, Indonesia",Tolotangga,105.58,4.4,mb,2016,NaN,NaN
5,2016-12-30 06:06:34.030000+00:00,0.4049,123.7758,"23km W of Molibagu, Indonesia",Molibagu,211.57,4.6,mb,2016,NaN,NaN
6,2016-12-29 22:30:19.300000+00:00,-9.0279,118.6639,"29km S of Tolotangga, Indonesia",Tolotangga,79.00,6.3,mww,2016,NaN,NaN
7,2016-12-29 14:37:33.160000+00:00,3.9723,126.7909,"216km SE of Sarangani, Philippines",Sarangani,82.30,4.3,mb,2016,NaN,NaN
8,2016-12-29 08:16:44.850000+00:00,-4.6133,101.7366,"107km SSW of Bengkulu, Indonesia",Bengkulu,31.98,4.3,mb,2016,NaN,NaN
9,2016-12-29 01:22:26.280000+00:00,-4.7053,101.6970,"118km SSW of Bengkulu, Indonesia",Bengkulu,31.82,4.3,mb,2016,NaN,NaN


In [15]:
fixkolom = fixkolom.dropna()
fixkolom

,time,latitude,longitude,place,city,depth,mag,magType,year,a-value,b-value
50,2016-12-18 01:45:21.140000+00:00,2.1700,99.0849,"50km E of Onan Ganjang, Indonesia",Onan Ganjang,148.66,4.1,mb,2016,4.156393,0.641744
51,2016-12-18 01:44:47.510000+00:00,0.7300,126.0984,"129km SE of Bitung, Indonesia",Bitung,45.15,4.8,mb,2016,4.161117,0.642577
52,2016-12-17 08:31:23.320000+00:00,5.5984,126.3710,"87km SSE of Pondaguitan, Philippines",Pondaguitan,70.20,4.2,mb,2016,4.152536,0.642692
53,2016-12-17 06:21:18.420000+00:00,3.4914,128.7740,"212km NNE of Tobelo, Indonesia",Tobelo,41.09,4.2,mb,2016,4.128367,0.638991
54,2016-12-17 03:02:09.090000+00:00,5.3424,125.4744,"6km S of Sarangani, Philippines",Sarangani,188.63,4.6,mb,2016,4.091461,0.636429
...,...,...,...,...,...,...,...,...,...,...,...
51144,1966-03-08 05:41:06+00:00,1.9030,126.3340,Molucca Sea,Molucca Sea,35.00,6.2,mw,1966,6.385604,0.846806
51145,1966-02-13 06:36:01+00:00,-6.5610,132.6420,"Kepulauan Tanimbar, Indonesia",Kepulauan Tanimbar,35.00,6.0,mw,1966,6.427646,0.852336
51146,1966-02-09 07:18:50+00:00,-9.8510,116.3760,"Lombok region, Indonesia",Lombok region,45.00,5.8,mw,1966,6.463244,0.857352
51147,1966-02-03 05:48:11+00:00,0.2110,123.5550,"Minahasa, Sulawesi, Indonesia",Minahasa,158.60,5.7,mw,1966,6.429364,0.853431


In [16]:
# Groupby & multiple aggregations
result = fixkolom.groupby('year')['a-value'].aggregate(['mean','min','max','sum'])

print(result)

          mean       min        max           sum
year                                             
1966  6.331350  5.982847   6.792293    120.295642
1967  5.961865  5.430380   6.837602    184.817808
1968  6.139311  5.454160   6.767150    239.433118
1969  7.188421  6.131092   8.055189    316.290524
1970  6.032018  5.761778   6.230088    211.120647
1971  6.054123  5.779578   6.584516    308.760292
1972  5.161957  4.268709   6.378586    320.041308
1973  6.237658  5.568858   7.786404   1596.840512
1974  5.717177  4.552321   9.195811   1863.799779
1975  5.640962  4.736490   7.231069   1974.336696
1976  5.392226  4.294617   6.572898   2162.282766
1977  5.764932  4.718941   8.127639   2697.988381
1978  5.876208  4.674712   7.097571   2080.177766
1979  5.080129  3.737830   6.486890   2509.583905
1980  5.582895  4.529110   7.105849   2607.212153
1981  5.458229  4.540690   6.340705   2467.119534
1982  5.303301  4.460470   6.159984   2699.380170
1983  5.446681  3.777811   7.829674   4085.010666


In [17]:
a_tot = fixkolom.groupby("year")["a-value"].sum()

a_ave = result["sum"].mean()

#**Spasial variation using values-a**

In [18]:
import geopandas as gpd

# Load GeoJSON data
geojson_url = "https://raw.githubusercontent.com/arifadli/DatasetRepository/main/Disertation/Indonesia/indonesia-province.json"
indonesia_map = gpd.read_file(geojson_url)

indonesia_map.head(10)

,ID,kode,Propinsi,SUMBER,geometry
0,1,85,IRIAN JAYA TIMUR,Peta Dasar BAKOSURTANAL Skala 1 : 250.000,"MULTIPOLYGON (((137.91666 -1.49852, 137.89260 ..."
1,2,52,NUSATENGGARA BARAT,Peta Dasar BAKOSURTANAL Skala 1 : 250.000,"MULTIPOLYGON (((117.62720 -8.50640, 117.62630 ..."
2,3,75,GORONTALO,Peta Dasar BAKOSURTANAL Skala 1 : 250.000,"MULTIPOLYGON (((122.18814 1.04530, 122.18883 1..."
3,4,74,SULAWESI TENGGARA,Peta Dasar BAKOSURTANAL Skala 1 : 250.000,"MULTIPOLYGON (((120.98423 -2.83534, 120.98502 ..."
4,5,34,DAERAH ISTIMEWA YOGYAKARTA,Peta Dasar BAKOSURTANAL Skala 1 : 250.000,"POLYGON ((110.01183 -7.88690, 110.01330 -7.885..."
5,6,3329,JAWA TENGAH,Peta Dasar BAKOSURTANAL Skala 1 : 250.000,"MULTIPOLYGON (((108.82934 -6.74608, 108.83061 ..."
6,7,36,PROBANTEN,Peta Dasar BAKOSURTANAL Skala 1 : 250.000,"MULTIPOLYGON (((106.72134 -6.09000, 106.71294 ..."
7,8,35,JAWA TIMUR,Peta Dasar BAKOSURTANAL Skala 1 : 250.000,"MULTIPOLYGON (((111.69460 -6.75286, 111.69649 ..."
8,9,81,MALUKU UTARA,Peta Dasar BAKOSURTANAL Skala 1 : 250.000,"MULTIPOLYGON (((127.74207 0.81629, 127.74280 0..."
9,10,82,MALUKU,Peta Dasar BAKOSURTANAL Skala 1 : 250.000,"MULTIPOLYGON (((130.83130 -3.47141, 130.83050 ..."


In [25]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [34]:
import requests
import geopandas as gpd
from zipfile import ZipFile
from io import BytesIO

# URL file zip yang berisi file GeoJSON
zip_url = "https://github.com/pararawendy/border-desa-indonesia-geojson/raw/master/indonesia_villages_border.geojson.zip"

# Unduh file zip
response = requests.get(zip_url)

# Baca file zip sebagai BytesIO object
zip_bytes = BytesIO(response.content)

# Buka file zip
with ZipFile(zip_bytes, "r") as zip_file:
    # Cari nama file GeoJSON di dalam zip
    file_list = zip_file.namelist()
    geojson_filename = None
    for file_name in file_list:
        if file_name.endswith('.geojson'):
            geojson_filename = file_name
            break

    # Jika file GeoJSON ditemukan
    if geojson_filename:
        # Baca file GeoJSON dari dalam zip
        with zip_file.open(geojson_filename) as geojson_file:
            # Baca file GeoJSON menggunakan GeoPandas
            gdf = gpd.read_file(geojson_file)
            # Tampilkan informasi singkat tentang GeoDataFrame
            print(gdf.head())
    else:
        print("File GeoJSON tidak ditemukan dalam file zip.")


ERROR:fiona._env:`/vsimem/6a945c67a23f495d83e7cefc8752e32f' not recognized as a supported file format.


DriverError: '/vsimem/6a945c67a23f495d83e7cefc8752e32f' not recognized as a supported file format.

In [19]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt

# Load GeoJSON data
geojson_url = "https://raw.githubusercontent.com/arifadli/DatasetRepository/main/Disertation/Indonesia/indonesia_villages_border.geojson"
indonesia_map = gpd.read_file(geojson_url)

# Extract city names from the 'place' column
fixkolom['city'] = fixkolom['place'].str.split(',').str[0].str.split('of').str[-1].str.strip()

# Merge the GeoJSON data with your DataFrame
merged_data = indonesia_map.merge(fixkolom, how='left', left_on='city', right_on='propinsi')

# Plot the map
fig, ax = plt.subplots(1, 1, figsize=(15, 10))

# Differentiate areas by color
indonesia_map.plot(ax=ax, color='white', edgecolor='black')
merged_data.plot(ax=ax, cmap='viridis', edgecolor='red', linewidth=1, legend=True)

# Add city names as legend
for x, y, label in zip(merged_data.geometry.centroid.x, merged_data.geometry.centroid.y, merged_data['propinsi']):
    ax.text(x, y, label, fontsize=8, ha='center', va='center')

# Add labels and title
plt.title('Map of California with City Boundaries and Names')
plt.xlabel('Longitude')
plt.ylabel('Latitude')

# Show the legend
plt.legend()

# Show the plot
plt.show()


KeyError: 'propinsi'

In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.legend_handler import HandlerPatch
from matplotlib.colors import Normalize
from mpl_toolkits.axes_grid1 import make_axes_locatable

# Load GeoJSON data
geojson_url = "https://raw.githubusercontent.com/arifadli/DatasetRepository/main/Disertation/California/california-counties.geojson"
california_map = gpd.read_file(geojson_url)

# Extract city names from the 'place' column
fixkolom['city'] = fixkolom['place'].str.split(',').str[0].str.split('of').str[-1].str.strip()

# Merge the GeoJSON data with your DataFrame
merged_data = california_map.merge(fixkolom, how='left', left_on='name', right_on='city')

# Handle NaN values in the 'a-value' column
merged_data['a-value'].fillna(merged_data['a-value'].mean(), inplace=True)

# Plot the map
fig, ax = plt.subplots(1, 1, figsize=(15, 10))

# Differentiate areas by color
cmap = 'viridis'
california_map.plot(ax=ax, color='white', edgecolor='black')
merged_data.plot(ax=ax, cmap=cmap, edgecolor='red', linewidth=1)

# Extract unique city names and their corresponding colors from the colormap
city_names = merged_data['name'].unique()

# Replace 'value' with the actual column name in your dataset
data_values = merged_data['a-value']  # Replace 'a-value' with your actual column name

# Normalize colormap to match data values
norm = Normalize(vmin=data_values.min(), vmax=data_values.max())
colors = plt.cm.get_cmap(cmap)(norm(data_values))

# Define a custom legend handler
class CityLegendHandler(HandlerPatch):
    def create_artists(self, legend, orig_handle, xdescent, ydescent, width, height, fontsize, trans):
        marker_size = 10
        color = orig_handle.get_markerfacecolor()
        marker = Line2D([0], [0], marker='o', color='w', markerfacecolor=color, markersize=marker_size)
        return [marker]

# Add city names and 'a-value' as custom legend
legend_elements = [Line2D([0], [0], marker='o', color='w', markerfacecolor=color, markersize=10,
                          label=f'{name} ({value:.2f})')
                   for name, color, value in zip(city_names, colors, data_values)]

ax.legend(handles=legend_elements, title='City Names', handler_map={Line2D: CityLegendHandler()}, ncol=4, loc='upper center', bbox_to_anchor=(1.5, 1))

# Add labels and title
plt.title('Map of California with City Boundaries and Names')
plt.xlabel('Longitude')
plt.ylabel('Latitude')

# Show the plot
plt.show()
